In [1]:
!curl -s https://course.fast.ai/setup/colab | bash
!pip install fire
%matplotlib inline
from fastai.basics import *

Updating fastai...
Done.
     |████████████████████████████████| 81kB 2.6MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=09cfbb56c5e128f67386add3712344ae2eedffd49d8344a237cfd217046c0dbf
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [3]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [4]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
x_train.shape,x_valid.shape

(torch.Size([50000, 784]), torch.Size([10000, 784]))

In [6]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [7]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [ ]:
nn.Conv2d?

In [13]:
x = x_valid[:100]

In [ ]:
x.shape

torch.Size([100, 1, 28, 28])

In [9]:
def stats(x): return x.mean(),x.std()

In [ ]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [ ]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0019, grad_fn=<MeanBackward0>),
  tensor(0.1152, grad_fn=<StdBackward0>)),
 (tensor(0.0105, grad_fn=<MeanBackward0>),
  tensor(0.0968, grad_fn=<StdBackward0>)))

In [19]:
t = l1(x)

In [20]:
t.shape

(torch.Size([100, 32, 24, 24]), torch.Size([100, 32, 18, 18]))

In [ ]:
stats(t)

(tensor(0.0107, grad_fn=<MeanBackward1>),
 tensor(0.5978, grad_fn=<StdBackward0>))

In [21]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

((tensor(-0.0379, grad_fn=<MeanBackward0>),
  tensor(1.1418, grad_fn=<StdBackward0>)),
 (tensor(-0.0086, grad_fn=<MeanBackward0>),
  tensor(1.2605, grad_fn=<StdBackward0>)))

In [22]:
import torch.nn.functional as F

In [25]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)


In [26]:
init.kaiming_normal_(l1.weight, a=0)

stats(f1(x))

((tensor(0.4715, grad_fn=<MeanBackward0>),
  tensor(0.9435, grad_fn=<StdBackward0>)),
 (tensor(0.6089, grad_fn=<MeanBackward0>),
  tensor(0.9783, grad_fn=<StdBackward0>)))

In [27]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2035, grad_fn=<MeanBackward0>),
 tensor(0.3824, grad_fn=<StdBackward0>))

In [31]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [32]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [33]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [34]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [45]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.)), gain(0)

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257,
 1.4142135623730951)

In [ ]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5788)

In [42]:
1/math.sqrt(3.0)

0.5783149319662402

In [43]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(2.99) * std
    x.data.uniform_(-bound,bound)

In [44]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5350, grad_fn=<MeanBackward0>),
 tensor(1.0693, grad_fn=<StdBackward0>))

In [46]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2190, grad_fn=<MeanBackward0>),
 tensor(0.4106, grad_fn=<StdBackward0>))

In [49]:
x.shape

torch.Size([100, 1, 28, 28])

In [50]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [51]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [52]:
y = y_valid[:100].float()

In [53]:
t = m(x)
stats(t)

(tensor(0.0112, grad_fn=<MeanBackward0>),
 tensor(0.0065, grad_fn=<StdBackward0>))

In [54]:
l = mse(t,y)
l.backward()

In [55]:
stats(m[0].weight.grad)

(tensor(-0.0003), tensor(0.0225))

In [ ]:
stats(m[0].weight.grad)

(tensor(0.0054), tensor(0.0333))

In [ ]:
init.kaiming_uniform_??

In [56]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [57]:
t = m(x)
stats(t)

(tensor(-0.5669, grad_fn=<MeanBackward0>),
 tensor(0.2284, grad_fn=<StdBackward0>))

In [58]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(-0.4355), tensor(0.5142))

## Export

In [59]:
!python notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py
